## Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install -U --pre tensorflow=="2.2.0"
!pip install -U --pre tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 49.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.6.3
    Uninstalling typing_extensions-4.6.3:
      Successfully uninstalled typing_extensions-4.6.3
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12

In [3]:
%cd /content/drive/MyDrive/comp472

/content/drive/MyDrive/comp472


In [ ]:
# Install the Object Detection API

%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/drive/MyDrive/comp472/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=3147434 sha256=8658614e58eca161f5ed2db0bff8bb6fc385654b73a39ab8b82e1425a5ad57e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7ee8iyc/wheels/9b/bb/31/06df0c9fbf2d0a4d34906b3b2dc886525c735a3d070ab1dd90
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [ ]:
!pip install -U numpy

  Using cached numpy-1.25.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.48.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.25.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.0 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.0 which is incompatible.


In [ ]:
import PIL
from scipy.spatial.distance import euclidean
from scipy.spatial import distance as dist

import PIL
from PIL import Image
import numpy as np

from imutils import perspective
from imutils import contours
import imutils
import cv2
from google.colab.patches import cv2_imshow
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Utilities

In [ ]:
def midpoint(ptA, ptB):
  return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def scale_calibration(num_pixel):
  pixel_per_metric = 24.5 / 166
  return num_pixel*pixel_per_metric

def k_means_segmentation(pil_image, im_show=False):

    # Convert to HSV format
    opencvImage = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    img_rgb = cv2.cvtColor(opencvImage,cv2.COLOR_BGR2RGB)
    img = cv2.cvtColor(img_rgb,cv2.COLOR_RGB2HSV)

    # reshape the image to a 2D array of pixels and 3 color values (RGB)
    pixel_values = img.reshape((-1, 3))
    # convert to float
    pixel_values = np.float32(pixel_values)
    # print(pixel_values.shape)

    # define stopping criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

    # number of clusters (K)
    k = 2
    _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 100, cv2.KMEANS_RANDOM_CENTERS)

    # convert back to 8 bit values
    centers = np.uint8(centers)

    # flatten the labels array
    labels = labels.flatten()

    # convert all pixels to the color of the centroids
    segmented_image = centers[labels.flatten()]

    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(img.shape)

    if im_show:
      # show the image
      plt.imshow(segmented_image)
      plt.show()
    return segmented_image, opencvImage

def apply_morphology(segmented_image, kernel, im_show=False):

    #Apply preprocessing to remove noise
    grayscale = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)   #converting the color to grayscale

    opening = cv2.morphologyEx(grayscale, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    # show the image
    if im_show:
        plt.imshow(closing)
        plt.show()
    return closing

def show_data_point_of_curve_polygon(img_rgb, closing_img):

    #  Python code to find the co-ordinates of the contours detected in an image.
    imgrgb = img_rgb.copy()
    # Reading image
    font = cv2.FONT_HERSHEY_COMPLEX

    # Converting image to a binary image
    # ( black and white only image).
    _, threshold = cv2.threshold(closing_img.copy(), 110, 255, cv2.THRESH_BINARY)
    # plt.imshow(threshold)
    # Detecting contours in image.
    contours, _= cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Going through every contours found in the image.
    for cnt in contours :

        area = cv2.contourArea(cnt)
        if area > 1000:


            approx = cv2.approxPolyDP(cnt, 0.010 * cv2.arcLength(cnt, True), True)

            # draws boundary of contours.
            cv2.drawContours(imgrgb, [approx], 0, (0, 0, 255), 1)

            # Used to flatted the array containing
            # the co-ordinates of the vertices.
            n = approx.ravel()
            i = 0

            for j in n :
                if(i % 2 == 0):
                    x = n[i]
                    y = n[i + 1]

                    # String containing the co-ordinates.
                    string = str(x) + " " + str(y)

                    if(i == 0):
                        # text on topmost co-ordinate.
                        cv2.putText(imgrgb, "Arrow tip", (int(x), int(y)),font, 0.4, (255, 0, 0))
                    else:
                        # text on remaining co-ordinates.
                        cv2.putText(imgrgb, string, (int(x), int(y)),
                                  font, 0.4, (0, 0, 255))
                i = i + 1

    # Showing the final image.
    cv2_imshow(imgrgb)

    # Exiting the window if 'q' is pressed on the keyboard.
    if cv2.waitKey(0) & 0xFF == ord('q'):
        cv2.destroyAllWindows()

def show_dimentionality_polygon(img_rgb, closing_img):


    # Reading image
    font = cv2.FONT_HERSHEY_COMPLEX
    imgrgb = img_rgb.copy()
    # imgrgb2 = img_rgb.copy()
    ret, thresh = cv2.threshold(closing_img.copy(), 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by area
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    n_detected_c = len(sorted_contours)
    print("number of detected contours :", n_detected_c)

    # number of desired contours
    n_desired_c = 2

    # comparison
    if n_detected_c >= n_desired_c:
      up_to = n_desired_c
    else :
      up_to = n_detected_c

    k = 0
    for c in sorted_contours[:up_to]:


                perimeter = cv2.arcLength(c, True)
                perimeter = round(perimeter, 4)
                area = cv2.contourArea(c)
                # print('Area:', area)
                # print('Perimeter:', perimeter)

                # x1, y1 = c[0,0]
                # cv2.putText(imgrgb, f'Area:{area}', (x1-100, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                # cv2.putText(imgrgb, f'Perimeter:{perimeter}', (x1-100, y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                # cv2.putText(imgrgb, "original contour, num_pts={}".format(len(c)), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                #----------------------------------------------------
                # Draw a Blue rectangle shaped boundiong box around circle
                #----------------------------------------------------

                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(imgrgb, (x, y), (x + w, y + h), (255, 0,0), 1)

                if k == 0:
                    cv2.putText(imgrgb, f'width of rectangle :{round(scale_calibration(w), 2)} mm', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    cv2.putText(imgrgb, f'hiegth of rectangle :{round(scale_calibration(h),2)} mm', (x, y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                #----------------------------------------------------
                # Draw a Yellow outline around the contour
                #----------------------------------------------------
                cv2.drawContours(imgrgb, [c], -1, (0,255,255), 2)

                #----------------------------------------------------
                # calculate center and radius of minimum enclosing circle
                # and draw circle pink shaped bounding box around circle
                #----------------------------------------------------
                (x_center,y_center) ,radius = cv2.minEnclosingCircle(c)
                # cast to integers
                center = (int(x_center),int(y_center))
                radius = int(radius)
                cv2.circle(imgrgb, center, radius, (255, 0, 255), 1)

                if k == 0 :
                    cv2.putText(imgrgb, f'Diameter of bigger circle :{round(scale_calibration(radius*2), 2)} mm', (center[0]-40, center[1]-200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    cv2.putText(imgrgb, f'Radius of bigger circle :{round(scale_calibration(radius), 2)} mm', (center[0]-40, center[1]-180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                    #----------------------------------------------------
                    # visuallize_data_point_of_curve_polygon and mesure the dimention
                    #----------------------------------------------------
                    # approx = cv2.approxPolyDP(c, 0.015 * cv2.arcLength(c, True), True)

                    # # draws boundary of contours.
                    # cv2.drawContours(imgrgb2, [approx], 0, (0, 0, 255), 1)

                    # # Used to flatted the array containing
                    # # the co-ordinates of the vertices.
                    # n = approx.ravel()
                    # i = 0

                    # for j in n :
                    #     if(i % 2 == 0):
                    #         x = n[i]
                    #         y = n[i + 1]

                    #         # String containing the co-ordinates.
                    #         string = str(x) + " " + str(y)

                    #         if(i == 0):
                    #             # text on topmost co-ordinate.
                    #             cv2.putText(imgrgb2, "Arrow tip", (int(x), int(y)),font, 0.4, (255, 0, 0))
                    #         else:
                    #             # text on remaining co-ordinates.
                    #             cv2.putText(imgrgb2, string, (int(x), int(y)),
                    #                       font, 0.4, (0, 0, 255))
                    #     i = i + 1


                else :
                    cv2.putText(imgrgb, f'Diameter:{round(scale_calibration(radius*2), 2)} mm', (center[0], center[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    cv2.putText(imgrgb, f'Radius:{round(scale_calibration(radius), 2)} mm', (center[0], center[1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

                k += 1


    # show the original contour image
    cv2_imshow(imgrgb)
    # cv2_imshow(imgrgb2)

def show_dimentionality_rectangle(img_rgb, closing_img):


    imgrgb = img_rgb.copy()
    # Reading image
    font = cv2.FONT_HERSHEY_COMPLEX

    ret, thresh = cv2.threshold(closing_img.copy(), 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort the contours by area
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    cordinate = []
    for cnt in sorted_contours :

        area = cv2.contourArea(cnt)
        if area >= cv2.contourArea(sorted_contours[0]):

            perimeter = cv2.arcLength(cnt, True)
            perimeter = round(perimeter, 2)

            #----------------------------------------------------
            # Draw a Yellow outline around the contour
            #----------------------------------------------------
            cv2.drawContours(imgrgb, [cnt], -1, (0,255,255), 2)

            #----------------------------------------------------
            # Draw a Blue rectangle shaped boundiong box around object
            #----------------------------------------------------
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(imgrgb, (x, y), (x + w, y + h), (255, 0,0), 1)

            #----------------------------------------------------
            # Draw a Red rectangle boundiong box generated from approximated data point of the curve
            #----------------------------------------------------
            approx = cv2.approxPolyDP(cnt, 0.002 * cv2.arcLength(cnt, True), True)

            # draws boundary of contours.
            cv2.drawContours(imgrgb, [approx], 0, (0, 0, 255), 1)

            # Used to flatted the array containing
            # the co-ordinates of the vertices.
            n = approx.ravel()
            i = 0

            for j in n :
                if(i % 2 == 0):
                    x = n[i]
                    y = n[i + 1]

                    # String containing the co-ordinates.
                    string = str(x) + " " + str(y)
                    cordinate.append((x,y))

                    print("approximated data point of the curve in term of pixel:", x,y,"x,y")
                    if(i == 0):
                        # text on topmost co-ordinate.
                        cv2.putText(imgrgb, string, (int(x-10), int(y-10)),font, 0.4, (0, 0, 255))
                    else:
                        # text on remaining co-ordinates.
                        cv2.putText(imgrgb, string, (int(x-10), int(y-10)),
                                  font, 0.4, (0, 0, 255))
                i = i + 1

    # Showing the final image.

    center = midpoint(midpoint(cordinate[0], cordinate[1]), midpoint(cordinate[2], cordinate[3]))
    # compute the Euclidean distance between the midpoints
    dA = dist.euclidean(cordinate[0], cordinate[1])
    cv2.putText(imgrgb, f'hiegth:{round(scale_calibration(dA), 2)}', (int(center[0]),int(center[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    dB = dist.euclidean(cordinate[1], cordinate[2])
    cv2.putText(imgrgb, f'width:{round(scale_calibration(dB),2)}', (int(center[0]),int(center[1])+50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    x1, y1 = cnt[0,0]  # to determine the coordinates
    area_of_rectangle = round(scale_calibration(dB),2) * round(scale_calibration(dA), 2)
    cv2.putText(imgrgb, f'Area:{area_of_rectangle} mm^2', (x1+int(center[0]), y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    perimeter_of_rectangle = (round(scale_calibration(dB),2) + round(scale_calibration(dA), 2)) * 2
    cv2.putText(imgrgb, f'Perimeter:{perimeter} mm', (x1+int(center[0]), y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    cv2_imshow(imgrgb)



def show_dimentionality_circle(img_rgb, closing_img):

    # Reading image
    font = cv2.FONT_HERSHEY_COMPLEX
    imgrgb = img_rgb.copy()
    ret, thresh = cv2.threshold(closing_img.copy(), 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours by area
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
    n_detected_c = len(sorted_contours)
    print("number of detected contours :", n_detected_c)

    # number of desired contours
    n_desired_c = 7

    # comparison
    if n_detected_c >= n_desired_c:
      up_to = n_desired_c
    else :
      up_to = n_detected_c
    i = 0
    for c in sorted_contours[:up_to]:


                perimeter = cv2.arcLength(c, True)
                perimeter = round(perimeter, 4)
                area = cv2.contourArea(c)
                # print('Area:', area)
                # print('Perimeter:', perimeter)

                # x1, y1 = c[0,0]
                # cv2.putText(imgrgb, f'Area:{area}', (x1-100, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                # cv2.putText(imgrgb, f'Perimeter:{perimeter}', (x1-100, y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                # cv2.putText(imgrgb, "original contour, num_pts={}".format(len(c)), (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                #----------------------------------------------------
                # Draw a Blue rectangle shaped boundiong box around circle
                #----------------------------------------------------

                x,y,w,h = cv2.boundingRect(c)
                cv2.rectangle(imgrgb, (x, y), (x + w, y + h), (255, 0,0), 1)

                #----------------------------------------------------
                # Draw a Yellow outline around the contour
                #----------------------------------------------------
                cv2.drawContours(imgrgb, [c], -1, (0,255,255), 2)

                #----------------------------------------------------
                # calculate center and radius of minimum enclosing circle
                # and draw circle pink shaped bounding box around circle
                #----------------------------------------------------
                (x_center,y_center) ,radius = cv2.minEnclosingCircle(c)
                # cast to integers
                center = (int(x_center),int(y_center))
                radius = int(radius)
                cv2.circle(imgrgb, center, radius, (255, 0, 255), 1)

                if i == 0 :
                    cv2.putText(imgrgb, f'Diameter of bigger circle :{round(scale_calibration(radius*2), 2)} mm', (center[0]-40, center[1]-200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    cv2.putText(imgrgb, f'Radius of bigger circle :{round(scale_calibration(radius), 2)} mm', (center[0]-40, center[1]-180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                else :
                    cv2.putText(imgrgb, f'Diameter:{round(scale_calibration(radius*2), 2)} mm', (center[0], center[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)
                    cv2.putText(imgrgb, f'Radius:{round(scale_calibration(radius), 2)} mm', (center[0], center[1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 255), 1)

                i += 1


    # show the original contour image
    cv2_imshow(imgrgb)


In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)


#recover our saved Tensorflow model

In [ ]:
%cd /content/drive/MyDrive/comp472

/content/drive/MyDrive/comp472


In [ ]:
%ls 'new_config/cheakpoint'

checkpoint  ckpt-1.data-00000-of-00001  ckpt-1.index


load the saved model weight

In [ ]:
import pathlib

filenames = list(pathlib.Path('new_config/cheakpoint').glob('*.index'))
pipeline_file = '/content/drive/MyDrive/comp472/new_config/pipeline.config'

filenames.sort()
print(filenames)

pipeline_config = pipeline_file
#generally you want to put the last ckpt from training in here
model_dir = str(filenames[-1]).replace('.index','')
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(str(filenames[-1]).replace('.index','')))


def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

[PosixPath('new_config/cheakpoint/ckpt-1.index')]


# Detect single object

Upload images

In [ ]:
%cd /content/drive/MyDrive/comp472
%rm -r /content/drive/MyDrive/comp472/TEST
os.makedirs("TEST",exist_ok=True)
%cd /content/drive/MyDrive/comp472/TEST
os.makedirs("new_img_for_test",exist_ok=True)
os.makedirs("resized_new_img_for_test",exist_ok=True)
os.makedirs("cropped",exist_ok=True)
%cd /content/drive/MyDrive/comp472/TEST/new_img_for_test


from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

/content/drive/MyDrive/comp472
rm: cannot remove '/content/drive/MyDrive/comp472/TEST': No such file or directory
/content/drive/MyDrive/comp472/TEST
/content/drive/MyDrive/comp472/TEST/new_img_for_test


Saving ANMP0076.jpg to ANMP0076.jpg
User uploaded file "ANMP0076.jpg" with length 1866123 bytes


resize the images

In [ ]:
%cd /content/drive/MyDrive/comp472
!python resize_images.py \
--raw-dir 'TEST/new_img_for_test' \
--save-dir 'TEST/resized_new_img_for_test' \
--ext jpg \
--target-size "(1888,1416)"

/content/drive/MyDrive/comp472
1 files to resize from directory `TEST/new_img_for_test` to target size:(1888, 1416)
.
Done resizing 1 files.
Saved to directory: `TEST/resized_new_img_for_test`


Inference from object detection model

In [ ]:
print("change directory to : ")
%cd /content/drive/MyDrive/comp472
label_id_offset =1

circle_CLASS_ID = 1
polygon_CLASS_ID = 2
rectangle_CLASS_ID = 3

category_index = {circle_CLASS_ID :
                     {'id'  : circle_CLASS_ID,'name': 'circle'},
                  polygon_CLASS_ID :
                      {'id'  : polygon_CLASS_ID,'name': 'polygon'},
                  rectangle_CLASS_ID :
                      {'id'  : rectangle_CLASS_ID,'name': 'rectangle'}
                  }

# we inputted the model the resized image...
test_img_dir = "/content/drive/MyDrive/comp472/TEST/resized_new_img_for_test"
out_dir = "/content/drive/MyDrive/comp472/TEST/cropped"

img_files = [x for x in os.listdir(test_img_dir) if x.endswith(('jpg', 'png'))]

for images in img_files:

    image_dir = os.path.join(test_img_dir, images)
    image = Image.open(image_dir)
    # image.show()

    np_image = np.array(image)
    (frame_height, frame_width) = np_image.shape[:2]

    input_tensor = tf.convert_to_tensor(np.expand_dims(np_image, 0), dtype=tf.float32)
    print("The resized shape of the tensor is :", input_tensor.shape)

    detections, predictions_dict, shapes = detect_fn(input_tensor)

    index_bbx = np.argmax(np.array(detections["detection_scores"][0]))
    y_min, x_min, y_max, x_max = detections["detection_boxes"][0][index_bbx]

    my_classes = detections['detection_classes'][0].numpy() + label_id_offset
    my_scores = detections['detection_scores'][0].numpy()

    min_score = 0.5

    object_name = [category_index[value]['name'] for index,value in enumerate(my_classes)  if my_scores[index] > min_score]
    print("The object is :", object_name[0])

    ymin = int((np.array(y_min)*frame_height))
    xmin = int((np.array(x_min)*frame_width))
    ymax = int((np.array(y_max)*frame_height))
    xmax = int((np.array(x_max)*frame_width))

    # Perform the cropping operation
    cropped_img = np_image[ymin:ymax,xmin:xmax]
    cropped_img = Image.fromarray(cropped_img)
    width, height = cropped_img.size

    # cropped_img.show()
    cropped_img.save(os.path.join(out_dir, f"cropped_{str(images)}"))

    if object_name[0] == "polygon":
      segmented_cropped_image, opencvImage = k_means_segmentation(cropped_img,im_show=False)
      closing_img = apply_morphology(segmented_cropped_image, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(13,13)),im_show=False)
      show_dimentionality_polygon(opencvImage, closing_img)


    if object_name[0] == "rectangle":
      segmented_cropped_image, opencvImage = k_means_segmentation(cropped_img,im_show=False)
      closing_img = apply_morphology(segmented_cropped_image, cv2.getStructuringElement(cv2.MORPH_RECT,(17,17)),im_show=False)
      show_dimentionality_rectangle(opencvImage, closing_img)


    if object_name[0] == "circle":
      segmented_cropped_image, opencvImage = k_means_segmentation(cropped_img,im_show=False)
      closing_img = apply_morphology(segmented_cropped_image, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(13,13)),im_show=False)
      show_dimentionality_circle(opencvImage, closing_img)
